<a href="https://colab.research.google.com/github/m-umamaheshwari/Creating-Multi-Task-Models-with-Keras/blob/main/Creating_Multi_Task_Models_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%load_ext tensorboard

# import the required libraries

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import shutil

try:
    shutil.rmtree('logs')
except:
    pass

In [ ]:
# load the traning and testing data from the keras library

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# This function will create noice in the grayscale images with the predominate colour as either red or blue

def create_example(x, y):
    # choise the predominant colour as random
    c = np.random.randint(0, 2)

    # create a image with noice
    image = 0.5 * np.random.rand(28, 28, 3)

    # add the grayscale image into the noice image
    image[:, :, c] += 0.5 *x / 255
    return image, y, c

In [ ]:
# dictionary with colour information
colors = {0: 'red', 1 : "grean"}

# let's see an example
x, y, c = create_example(x_train[1], y_train[1])
print(f'digit : {y}, color: {colors[c]}')

plt.imshow(x)
plt.show()

In [ ]:
# This function will generate a batch at random
# This is a generator function, each time it is call, it will generate a batch at random

def generate_data(x, y, batch_size = 32):
    num_examples = len(y)

    while True :
        x_batch = np.zeros((batch_size, 28, 28, 3))
        y_batch = np.zeros((batch_size,))
        c_batch = np.zeros((batch_size,))

        for i in range(0, batch_size):
            index = np.random.randint(0, num_examples)
            image, digit, color = create_example(x[index], y[index])
            x_batch[i] = image
            y_batch[i] = digit
            c_batch[i] = color

        yield x_batch, [y_batch, c_batch]

In [ ]:
# generate an example

x, [y, c] = next(generate_data(x_test, y_test, 1))

In [ ]:
# show the result

print(y,c)
plt.imshow(x[0])

In [ ]:
# import the required layers
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPool2D, Flatten, Add, Dense

# First the input layer
input_ = Input(shape=(28,28,3), name = 'input')

# First convolution layer
conv_1 = Conv2D(32, 3,  name = 'conv_1')(input_)
# activation layer
act_1 = Activation('relu', name = 'act_1')(conv_1)

# Maxpool layer
pool_1 = MaxPool2D(4,name = 'pool_1')(act_1)
# Flatten layer
flat_1 = Flatten(name = 'flat_1')(pool_1)

# First Output Layer
color =  Dense(1,activation = 'sigmoid', name = 'color')(flat_1)

# Second convolution layer
conv_2 = Conv2D(32, 3, padding = 'same', name = 'conv_2')(act_1)
# activation layer
act_2 = Activation("relu", name='act_2')(conv_2)

# Third convolution layer
conv_3 = Conv2D(32, 3, padding = 'same', name = 'conv_3')(act_2)

# Add Layer :
# connect the first convolution layer's output with the output of the thired convolution layer and add both.
add = Add(name = 'add')([act_1, conv_3])

# activation layer
act_3 = Activation("relu", name='act_3')(add)

# maxpool layer
pool_2 = MaxPool2D(4, name = 'pool_2')(act_3)
# flatten layer
flat_2 = Flatten(name = "flat_2")(pool_2)

# Our Secound Output layers
digit = Dense(10, activation = 'softmax', name = 'digit')(flat_2)

# Create the model
model = tf.keras.models.Model(input_, [digit, color])

# Compile the model
model.compile(
              # since we have two output layer, we can define two loss function
              loss = {
                  'digit' : 'sparse_categorical_crossentropy',
                  'color' : 'binary_crossentropy'
              },
              optimizer = 'adam',
              metrics = ['accuracy']
)

# To view the model architecture
model.summary()

In [ ]:
# To View the neural network layout/connection

tf.keras.utils.plot_model(model)

In [ ]:
# Since we have two output layers, we have to use custom class to generate/print the log

class Logger(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        # get the output accuracy
        digit_accuracy = logs.get('digit_accuracy')
        color_accuracy = logs.get('color_accuracy')

        # get the verbos accuracy
        val_digit_accuracy = logs.get('val_digit_accuracy')
        val_color_accuracy = logs.get('val_color_accuracy')

        # print the log
        print('='*30, epoch + 1, '='*30)
        print(f'digit_accuracy: {digit_accuracy:.2f}, color_accuracy: {color_accuracy:.2f}')
        print(f'val_digit_accuracy: {val_digit_accuracy:.2f}, val_color_accuracy: {val_color_accuracy:.2f}')

In [ ]:
# This function will generate a batch at random
# This is a generator function, each time it is call, it will generate a batch at random

def generate_data(x, y, batch_size = 32):
    num_examples = len(y)

    while True :
        x_batch = np.zeros((batch_size, 28, 28, 3))
        y_batch = np.zeros((batch_size,))
        c_batch = np.zeros((batch_size,))

        for i in range(0, batch_size):
            index = np.random.randint(0, num_examples)
            image, digit, color = create_example(x[index], y[index])
            x_batch[i] = image
            y_batch[i] = digit
            c_batch[i] = color
        # Change: Yield a tuple instead of a list for the labels
        yield x_batch, (y_batch, c_batch)

In [ ]:
# this function will test the model on the test dataset

def test_model(show=True):
    # generate the test data (1 at a time)
    x, [y, c] = next(test)

    # predict the model on test batch
    preds = model.predict(x)
    pred_digit = np.argmax(preds[0])
    pred_color = int(preds[1] > 0.5)

    # actual output
    gt_digit = int(y[0])
    gt_color = int(c[0])

    # show/display the image
    plt.imshow(x[0])

    if show:
        print(f'GT: {gt_digit}, {colors[gt_color]}')
        print(f'Pr: {pred_digit}, {colors[pred_color]}')
        plt.show()
    else:
        # show with green colour if the prediction is correct
        # show with red colour if the prediction is wrong
        col = 'green' if gt_digit == pred_digit and gt_color == pred_color else 'red'
        plt.ylabel(f'GT: {gt_digit}, {colors[gt_color]}', color=col)
        plt.xlabel(f'Pr: {pred_digit}, {colors[pred_color]}', color=col)
        plt.xticks([])
        plt.yticks([])

test = generate_data(x_test, y_test, batch_size=1)

In [ ]:
test_model()

In [ ]:
# Generate in a 4x4 matrix

plt.figure(figsize = (16,16))
for i in range(16):
    plt.subplot(4,4,i+1)
    test_model(show = False)
plt.show()